In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [5]:
from qreservoirpy.reservoirs import Static
class CustomRes(Static):
    def before(self, circuit):
        circuit.h(circuit.qubits)
        circuit.barrier()

    def during(self, circuit, timestep):
        circuit.initialize(str(timestep), [0])
        circuit.h(0)
        circuit.cx(0, 1)

    def after(self, circuit):
        circuit.barrier()
        circuit.measure_all()

res = CustomRes(n_qubits=2)
fig = res.circuit([0, 1]).draw('mpl')
fig.savefig("Images/simple_static.jpg", bbox_inches='tight')

c:\users\otkul\sintef\repo\reservoircomputingexamples\qreservoirpy\qreservoirpy\reservoirbase.py:38: UserWarning: Argument 'backend' must inherit from qiskits abstract 'Backend'. Using 'aer_simulator' instead
  warnings.warn("Argument 'backend' must inherit from qiskits abstract 'Backend'. Using 'aer_simulator' instead")


In [3]:
from qreservoirpy import QReservoir, Layers
def build_method(circuit, timestep):
    circuit.barrier()
    circuit.measure(circuit.qubits)
    return circuit

res = QReservoir(qubits=4, layers=[
    Layers.H(),
    Layers.Timeseries(build_method=build_method)
])
res.run([0, 1, 2])
fig = res.circuit.draw('mpl')
fig.savefig("Images/simple_timeseries.jpg", bbox_inches='tight')

Simulating:   0%|          | 0/1 [00:00<?, ?it/s]

Simulating: 100%|██████████| 1/1 [00:01<00:00,  1.78s/it]


In [4]:
from qreservoirpy import QReservoir, Layers
from qiskit.quantum_info import random_unitary

def build_method(circuit, timestep, operators, encoder):
    circuit.unitary(operators[timestep], circuit.qubits)
    circuit.measure([0, 1])
    circuit.initialize(encoder[timestep], [0, 1])
    return circuit


timeseries = [0, 1, 2]
res = QReservoir(qubits=4, layers=[
    Layers.H(),
    Layers.Timeseries(build_method=build_method,
        operators = {key : random_unitary(2**4) for key in timeseries}, 
        encoder = {
            0: '00',
            1: '01',
            2: '10',
            3: '11'
        })
])
res.run(timeseries)
fig = res.circuit.draw('mpl')
fig.savefig("Images/adding_parameters.jpg", bbox_inches='tight')

Simulating: 100%|██████████| 1/1 [00:01<00:00,  1.70s/it]


In [5]:
from qreservoirpy import QReservoir, Layers
from qiskit.quantum_info import random_unitary

def build_method(circuit, timestep, operator):
    circuit.unitary(operator, circuit.qubits)
    return circuit


timeseries = [0, 1, 2]
res = QReservoir(qubits=4, layers=[
    Layers.H(),
    Layers.Timeseries(
        build_method=build_method, 

        operator = random_unitary(2**4)
        ),
    Layers.Measurement(range(4))
])
res.run(timeseries, incrementally=True)
fig = res.circuit.draw('mpl')
fig.savefig("Images/incrementally.jpg", bbox_inches='tight')

Simulating:   0%|          | 0/3 [00:00<?, ?it/s]

Simulating: 100%|██████████| 3/3 [00:01<00:00,  2.75it/s]


In [6]:
from qreservoirpy import QReservoir, Layers
from qiskit.quantum_info import random_unitary

def analyze_fcn(sim_result):
    return sim_result[-4:]

def build_method(circuit, timestep, operator):
    circuit.measure(0)
    circuit.unitary(operator, circuit.qubits)
    return circuit


timeseries = [0, 1, 2]
res = QReservoir(qubits=4, layers=[
    Layers.H(),
    Layers.Timeseries(
        build_method=build_method, 
        operator = random_unitary(2**4),
        ),
    Layers.Measurement(range(4))
], analyze_function=analyze_fcn)
res.run(timeseries, incrementally=True)
fig = res.circuit.draw('mpl')
fig.savefig("Images/nonlinear_incrementally.jpg", bbox_inches='tight')

Simulating: 100%|██████████| 3/3 [00:01<00:00,  1.53it/s]
